In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definir las rutas a los directorios de imágenes positivas y negativas
positive_dir = '/content/drive/MyDrive/Modelado/positivo'
negative_dir = '/content/drive/MyDrive/Modelado/negativo'
test_dir = '/content/drive/MyDrive/Modelado/Prueba'

# Otras configuraciones, como el tamaño de imagen, batch_size, etc.
img_width, img_height = 224, 224  # Ajusta el tamaño de imagen según tus necesidades
batch_size = 32  # Ajusta el tamaño de lote según tus necesidades
num_epochs = 30

# Crear generadores de datos con aumento para entrenamiento y validación.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalización, igual que en entrenamiento y validación
    rotation_range=10,  # Rotación más leve
    width_shift_range=0.1,  # Cambio de ancho menor
    height_shift_range=0.1,  # Cambio de alto menor
    shear_range=0.1,  # Menor efecto de deformación
    zoom_range=0.1,  # Menor rango de zoom
    horizontal_flip=False,  # Desactivar volteo horizontal
    fill_mode='nearest')  # Usar el método de relleno más cercano)

# Crear un generador de datos de flujo desde el directorio de prueba
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Usar el directorio de imágenes de prueba
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Usar class_mode='binary' para dos clases (positivo y negativo)
    interpolation='nearest',  # Puedes cambiar la interpolación según tus necesidades
    shuffle=False,  # Desactiva la aleatorización de datos para que los resultados sean coherentes
    classes=['positivo', 'negativo'])  # Especificar explícitamente los nombres de las clases

# Crear generadores de datos de flujo desde directorios para entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    positive_dir,  # Usar el directorio de imágenes positivas
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Usar class_mode='binary' para dos clases (positivo y negativo)
    interpolation='nearest',  # Puedes cambiar la interpolación según tus necesidades
    subset='training',  # Agregar esta línea para especificar que es el conjunto de entrenamiento
    classes=['class_positivo', 'class_negativo'])  # Especificar explícitamente los nombres de las clases


validation_generator = validation_datagen.flow_from_directory(
    negative_dir,  # Usar el directorio de imágenes negativas
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Usar class_mode='binary' para dos clases (positivo y negativo)
    interpolation='nearest',  # Puedes cambiar la interpolación según tus necesidades
    shuffle=False,  # Desactivar la aleatorización de datos para que los resultados sean coherentes
)




# Definir el modelo CNN con regularización Dropout
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Agregar regularización Dropout
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Número de pasos por época
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)  # Número de pasos de validación

# Evaluar el modelo en el conjunto de prueba (debes cargar los datos de prueba de manera similar a los datos de entrenamiento y validación)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Accuracy on test data: {test_acc}')

# Guardar el modelo entrenado para su posterior uso
model.save('spectral_banana_detection_model.h5')


Found 62 images belonging to 2 classes.
Found 52 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/30
1/1 [==============================] - 6s 6s/step - loss: 0.5931 - accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - 2s 2s/step - loss: 9.6041e-26 - accuracy: 1.0000
Epoch 3/30
1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/30
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/30
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/30
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
